# BioBERT für Relationsextraktion

## Step 1: Korpusdaten verarbeiten
Mit `lxml.etree` die Korpus-XML-Dateien verarbeiten.

Siehe: [https://lxml.de/tutorial.html](https://lxml.de/tutorial.html)

Am Ende geht es um die Paare, die eindeutig durch eine ID identifiziert werden (document ID, sentence ID, pair ID).

* `id`
* `label` bzw. `interaction`
* `sentence`
* `e1_span`
* `e2_span`

Beispielauszug aus dem BioInfer Korpus:

```xml
<corpus id="BioInfer">
  ...
  <document id="BioInfer.d1" origId="8001585">
    ...
    <sentence id="BioInfer.d1.s1" origId="235" text="Birch profilin increased the critical concentration required for muscle and brain muscl polymerization in a concentration-dependent manner, supporting the notion of the formation of a heterologous complex between the plant protein and animal actin.">
      <entity charOffset="76-86" id="BioInfer.d1.s1.e0" origId="e.235.4" text="brain actin" type="Individual_protein" />
      <entity charOffset="6-13" id="BioInfer.d1.s1.e1" origId="e.235.5" text="profilin" type="Individual_protein" />
      <entity charOffset="65-70,82-86" id="BioInfer.d1.s1.e2" origId="e.235.6" text="muscle actin" type="Individual_protein" />
      <entity charOffset="242-246" id="BioInfer.d1.s1.e3" origId="e.235.7" text="actin" type="Individual_protein" />
      <pair e1="BioInfer.d1.s1.e0" e2="BioInfer.d1.s1.e1" id="BioInfer.d1.s1.p0" interaction="True" />
      <pair e1="BioInfer.d1.s1.e0" e2="BioInfer.d1.s1.e2" id="BioInfer.d1.s1.p1" interaction="False" />
      <pair e1="BioInfer.d1.s1.e0" e2="BioInfer.d1.s1.e3" id="BioInfer.d1.s1.p2" interaction="False" />
      <pair e1="BioInfer.d1.s1.e1" e2="BioInfer.d1.s1.e2" id="BioInfer.d1.s1.p3" interaction="True" />
      <pair e1="BioInfer.d1.s1.e1" e2="BioInfer.d1.s1.e3" id="BioInfer.d1.s1.p4" interaction="True" />
      <pair e1="BioInfer.d1.s1.e2" e2="BioInfer.d1.s1.e3" id="BioInfer.d1.s1.p5" interaction="False" />
    </sentence>
    ...
  </document>
  ...
</corpus>
```




Problematisch sind überlappende `charOffset`s:

```xml
<entities>
    <entity charOffset="76-86" id="BioInfer.d1.s1.e0" origId="e.235.4" text="brain actin" type="Individual_protein" />
    <entity charOffset="65-70,82-86" id="BioInfer.d1.s1.e2" origId="e.235.6" text="muscle actin" type="Individual_protein" />
</entities>
```

Bei der Anonymisierung ersetzen wir den Teil mit der größeren Spanne durch `@PROTEIN$`.
Bei den Positionsmarkierung setzen wir die Marker normal ein.

In [1]:
import sys
sys.path.append("../")
from tlbiore.data import corpus_processor, utils
import pandas as pd

Einstellungen:

In [2]:
aimed_train = '../data/raw/AIMed-train.xml'
bioinfer_train = '../data/raw/BioInfer-train.xml'

# Testdateien, für die wir die Predictions machen
aimed_pred = '../data/raw/AIMed-test.xml'
bioinfer_pred = '../data/raw/BioInfer-test.xml'

data_path = '../data/ppi_hu/'

Wir verarbeiten die XML-Daten:

In [3]:
ppi_train_corpus = corpus_processor.process_corpora([aimed_train, bioinfer_train])
ppi_pred_corpus = corpus_processor.process_corpora([aimed_pred, bioinfer_pred])

Hier führen wir den Split aus:

In [4]:
ppi_split_objects = ppi_train_corpus.get_sentences()
train, dev, test = utils.train_dev_test_split(ppi_split_objects)
    
# Resampling 
mask = train.label == 1
train_interaction = train[mask]
train_no_interaction = train[~mask]

# Oversampling
oversampled_interaction = train_interaction.sample(len(train_no_interaction),replace=True)
train_os = pd.concat([oversampled_interaction, train_no_interaction])

# Undersampling
undersampled_no_interaction = train_no_interaction.sample(len(train_interaction))
train_us = pd.concat([train_interaction, undersampled_no_interaction])

In [ ]:
pred = ppi_pred_corpus.get_examples()

## Step 2: Vorbereitung für BERT

1. Unser Hauptpaper: **"A BERT-based Universal Model for Both Within- and Cross-sentence Clinical Temporal Relation Extraction"** markiert die Positionen der Entitäten durch spezielle Non-XML Tags.
Wir könnten zum Beispiel `ps` (protein start) und `pe` (protein end) verwenden.
2. Alibaba: **"Enriching Pre-trained Language Model with Entity Information for Relation Classification"** markieren die Position der ersten Entität durch $-Zeichen und die Position der zweiten Entität durch #-Zeichen. 
3. BioBERT: **"BioBERT: a pre-trained biomedical language representation model for biomedical text mining"** anonymisiert die Entitäten. Bei uns würde man die Entitäten einfach durch `@PROTEIN$` ersetzen. Evtl. muss man die Entitäten z.B. durch eine Zahl am Ende unterscheiden?

Um diese speziellen Tokens - `ps` (protein start) und `pe` (protein end) -zu verwenden, müssen wir unser BioBERT Vokabular anpassen.

**HOW?**

Jedenfalls müssen wir unsere Pandas DataFrames den Ansätzen entsprechend anpassen. Dann müssen wir die jeweils in Train, Dev, Test aufteilen und am als .jsonl-Dateien abspeichern, um später mit HuggingFace arbeiten zu können.

Hier für sollte reichen:

*   `id`
*   `sentence`
*   `label`


### 2.1 Ansätze 1 & 2: Markiere die Positionen der Entitäten

#### 2.1.1: Unser Hauptpaper: "A BERT-based Universal Model for Both Within- and Cross-sentence Clinical Temporal Relation Extraction" 
Hier markieren wir die Entitäten mit `ps` (protein start) und `pe` (protein end).
Wir müssten uns das Vokabular anschauen und evtl. etwas anpassen.

In [6]:
lin_train = corpus_processor.prepare_data_lin(train)
lin_dev = corpus_processor.prepare_data_lin(dev)
lin_test = corpus_processor.prepare_data_lin(test)

Exportieren:

In [7]:
utils.export_jsonl(lin_train, data_path + 'lin/train.jsonl')
utils.export_jsonl(lin_dev, data_path + 'lin/dev.jsonl')
utils.export_jsonl(lin_test, data_path + 'lin/test.jsonl')

Analog mit den "richtigen" Testdaten ohne Labels:

In [8]:
lin_pred = corpus_processor.prepare_data_lin(pred)
utils.export_jsonl(lin_pred, data_path + 'lin/predict.jsonl', with_label=False)

Resampling

In [9]:
# Resampled train
lin_train_os = corpus_processor.prepare_data_lin(train_os)
lin_train_us = corpus_processor.prepare_data_lin(train_us)
utils.export_jsonl(lin_train_os, data_path + 'lin/train_os.jsonl')
utils.export_jsonl(lin_train_us, data_path + 'lin/train_us.jsonl')

#### 2.1.2: Alibaba: "Enriching Pre-trained Language Model with Entity Information for Relation Classification" 
Sie markieren die Position der ersten Entität durch `$`-Zeichen und die Position der zweiten Entität durch `#`-Zeichen.


In [10]:
ali_train = corpus_processor.prepare_data_ali(train)
ali_dev = corpus_processor.prepare_data_ali(dev)
ali_test = corpus_processor.prepare_data_ali(test)

Exportieren:

In [11]:
utils.export_jsonl(ali_train, data_path + 'ali/train.jsonl')
utils.export_jsonl(ali_dev, data_path + 'ali/dev.jsonl')
utils.export_jsonl(ali_test, data_path + 'ali/test.jsonl')

Analog mit den "richtigen" Testdaten ohne Labels:

In [12]:
ali_pred = corpus_processor.prepare_data_ali(pred)
utils.export_jsonl(ali_pred, data_path + 'ali/predict.jsonl', with_label=False)

Resampling

In [13]:
# Resampled train
ali_train_os = corpus_processor.prepare_data_ali(train_os)
ali_train_us = corpus_processor.prepare_data_ali(train_us)
utils.export_jsonl(ali_train_os, data_path + 'ali/train_os.jsonl')
utils.export_jsonl(ali_train_us, data_path + 'ali/train_us.jsonl')

### 2.2: BioBERT: "BioBERT: a pre-trained biomedical language representation model for biomedical text mining" 
Sie anonymisieren die Entitäten. Bei uns würde man die Entitäten einfach durch `@PROTEIN$` ersetzen.

In [14]:
lee_train = corpus_processor.prepare_data_lee(train)
lee_dev = corpus_processor.prepare_data_lee(dev)
lee_test = corpus_processor.prepare_data_lee(test)

Exportieren:

In [15]:
utils.export_jsonl(lee_train, data_path + 'lee/train.jsonl')
utils.export_jsonl(lee_dev, data_path + 'lee/dev.jsonl')
utils.export_jsonl(lee_test, data_path + 'lee/test.jsonl')

Analog mit den "richtigen" Testdaten ohne Labels:

In [16]:
lee_pred = corpus_processor.prepare_data_lee(pred)
utils.export_jsonl(lee_pred, data_path + 'lee/predict.jsonl', with_label=False)

Resampling

In [17]:
# Resampled train
lee_train_os = corpus_processor.prepare_data_lee(train_os)
lee_train_us = corpus_processor.prepare_data_lee(train_us)
utils.export_jsonl(lee_train_os, data_path + 'lee/train_os.jsonl')
utils.export_jsonl(lee_train_us, data_path + 'lee/train_us.jsonl')